## TUTORIAL 04 - Graetz problem 1
**_Keywords: successive constraints method_**

### 1. Introduction
This Tutorial addresses geometrical parametrization and the successive constraints method (SCM). In particular, we will solve the Graetz problem, which deals with forced heat convection in a channel $\Omega_o(\mu_0)$ divided into two parts $\Omega_o^1$ and $\Omega_o^2(\mu_0)$, as in the following picture:

<img src="data/graetz_1.png" width="70%"/>

Boundaries $\Gamma_{o, 1} \cup \Gamma_{o, 5} \cup \Gamma_{o, 6}$ are kept at low temperature (say, zero), while boundaries $\Gamma_{o, 2}(\mu_0) \cup \Gamma_{o, 4}(\mu_0)$ are kept at high temperature (say, one). The convection is characterized by the velocity $\boldsymbol{\beta} = (x_1(1-x_1), 0)$, being $\boldsymbol{x}_o = (x_{o, 0}, x_1)$ the coordinate vector on the parametrized domain $\Omega_o(\mu_0)$.

The problem is characterized by two parameters. The first parameter $\mu_0$ controls the shape of deformable subdomain $\Omega_2(\mu_0)$. The heat transfer between the domains can be taken into account by means of the Péclet number, which will be labeled as the parameter $\mu_1$. The ranges of the two parameters are the following:
$$\mu_0 \in [0.1,10.0] \quad \text{and} \quad \mu_1 \in [0.01,10.0].$$

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0, \mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,10.0]\times[0.01,10.0].
$$

In order to obtain a faster (yet, provably accurate) approximation of the problem, and avoiding _any_ remeshing, we pursue a model reduction by means of a certified reduced basis reduced order method from a fixed reference domain.
The successive constraints method will be used to evaluate the stability factors.

### 2. Parametrized formulation

Let $u_o(\boldsymbol{\mu})$ be the temperature in the domain $\Omega_o(\mu_0)$.

We will directly provide a weak formulation for this problem
<center>for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u_o(\boldsymbol{\mu})\in\mathbb{V}_o(\boldsymbol{\mu})$ such that</center>

$$a_o\left(u_o(\boldsymbol{\mu}),v_o;\boldsymbol{\mu}\right)=f_o(v_o;\boldsymbol{\mu})\quad \forall v_o\in\mathbb{V}_o(\boldsymbol{\mu})$$

where

* the function space $\mathbb{V}_o(\boldsymbol{\mu})$ is defined as
$$
\mathbb{V}_o(\mu_0) = \left\{ v \in H^1(\Omega_o(\mu_0)): v|_{\Gamma_{o,1} \cup \Gamma_{o,5} \cup \Gamma_{o,6}} = 0, v|_{\Gamma_{o,2}(\mu_0) \cup \Gamma_{o,2}(\mu_0)} = 1 \right\}
$$
Note that, as in the previous tutorial, the function space is parameter dependent due to the shape variation. 
* the parametrized bilinear form $a_o(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V}_o(\boldsymbol{\mu}) \times \mathbb{V}_o(\boldsymbol{\mu}) \to \mathbb{R}$ is defined by
$$a_o(u_o,v_o;\boldsymbol{\mu}) = \mu_1\int_{\Omega_o(\mu_0)} \nabla u_o \cdot \nabla v_o \ d\boldsymbol{x} + \int_{\Omega_o(\mu_0)} x_1(1-x_1) \partial_{x} u_o\ v_o \ d\boldsymbol{x},$$
* the parametrized linear form $f_o(\cdot; \boldsymbol{\mu}): \mathbb{V}_o(\boldsymbol{\mu}) \to \mathbb{R}$ is defined by
$$f_o(v_o;\boldsymbol{\mu}) = 0.$$

The successive constraints method will be used to compute the stability factor of the bilinear form $a_o(\cdot, \cdot; \boldsymbol{\mu})$.

In [1]:
import sys
sys.path.append("../../MLniCS/")
from mlnics import NN, Losses, Normalization
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(0)
np.random.seed(0)

## 3. Affine decomposition

In order to obtain an affine decomposition, we proceed as in the previous tutorial and recast the problem on a fixed, parameter _independent_, reference domain $\Omega$. As reference domain which choose the one characterized by $\mu_0 = 1$ which we generate through the generate_mesh notebook provided in the _data_ folder.
As in the previous tutorial, we pull back the problem to the reference domain $\Omega$.

In [2]:
@SCM()
@PullBackFormsToReferenceDomain()
@ShapeParametrization(
    ("x[0]", "x[1]"),  # subdomain 1
    ("mu[0]*(x[0] - 1) + 1", "x[1]"),  # subdomain 2
)
class Graetz(EllipticCoerciveProblem):

    # Default initialization of members
    @generate_function_space_for_stability_factor
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        EllipticCoerciveProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=subdomains)
        self.ds = Measure("ds")(subdomain_data=boundaries)
        # Store the velocity expression
        self.vel = Expression("x[1]*(1-x[1])", element=self.V.ufl_element())
        # Customize eigen solver parameters
        self._eigen_solver_parameters.update({
            "bounding_box_minimum": {
                "problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                "spectral_shift": 1.e-5, "linear_solver": "mumps"
            },
            "bounding_box_maximum": {
                "problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                "spectral_shift": 1.e5, "linear_solver": "mumps"
            },
            "stability_factor": {
                "problem_type": "gen_hermitian", "spectral_transform": "shift-and-invert",
                "spectral_shift": 1.e-5, "linear_solver": "mumps"
            }
        })

    # Return custom problem name
    def name(self):
        return "Graetz1"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_stability_factor
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = mu[1]
            theta_a1 = 1.0
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = 1.0
            return (theta_f0,)
        elif term == "dirichlet_bc":
            theta_bc0 = 1.0
            return (theta_bc0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_stability_factor
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            u = self.u
            vel = self.vel
            a0 = inner(grad(u), grad(v)) * dx
            a1 = vel * u.dx(0) * v * dx
            return (a0, a1)
        elif term == "f":
            f0 = Constant(0.0) * v * dx
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 1),
                   DirichletBC(self.V, Constant(1.0), self.boundaries, 2),
                   DirichletBC(self.V, Constant(1.0), self.boundaries, 4),
                   DirichletBC(self.V, Constant(0.0), self.boundaries, 5),
                   DirichletBC(self.V, Constant(0.0), self.boundaries, 6)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh_1.ipynb](data/generate_mesh_1.ipynb) notebook.

In [3]:
mesh = Mesh("data/graetz_1.xml")
subdomains = MeshFunction("size_t", mesh, "data/graetz_physical_region_1.xml")
boundaries = MeshFunction("size_t", mesh, "data/graetz_facet_region_1.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the Graetz class

In [5]:
problem = Graetz(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.1, 10.0), (0.01, 10.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a reduced basis method

In [6]:
reduction_method = ReducedBasis(problem)
reduction_method.set_Nmax(30, SCM=20)
reduction_method.set_tolerance(1e-5, SCM=1e-3)

### 4.5. Perform the offline phase

In [7]:
lifting_mu = (1.0, 1.0)
problem.set_mu(lifting_mu)
reduction_method.initialize_training_set(200, SCM=250)
reduced_problem = reduction_method.offline()

In [8]:
net = NN.RONN(problem, reduction_method)

reduction_method.initialize_testing_set(100, SCM=100)
test_mu, val_mu = NN.get_test_validation(net, validation_proportion=0.2)

In [9]:
input_normalization = Normalization.StandardNormalization()
output_normalization = Normalization.IdentityNormalization()

In [10]:
pdnn_loss = Losses.PDNN_Loss(net, output_normalization)
pinn_loss = Losses.PINN_Loss(net)
prnn_loss = Losses.PRNN_Loss(net, output_normalization, omega=1.)

In [11]:
NN.normalize_and_train(net, pinn_loss, input_normalization, 
                       lr=0.00001, epochs=10000, validation_set=val_mu)

Getting operator matrices...
Operator 's' not implemented. Continuing without operator 's'...
Operator 'stability_factor_left_hand_matrix' not implemented. Continuing without operator 'stability_factor_left_hand_matrix'...
Operator 'stability_factor_right_hand_matrix' not implemented. Continuing without operator 'stability_factor_right_hand_matrix'...
0 713.06676876933 	mean error(validation) = 1.049566996044788
100 164.22878856043067 	mean error(validation) = 1.0377206612946503
200 88.1504794379574 	mean error(validation) = 1.0214898160914792
300 55.50486924843459 	mean error(validation) = 1.0048777474963444
400 36.360893564564996 	mean error(validation) = 0.9919555028910402
500 24.68112087489137 	mean error(validation) = 0.9819736095101602
600 17.312969556156737 	mean error(validation) = 0.9740678692741234
700 12.545688748469745 	mean error(validation) = 0.9676112610399643
800 9.404465913643833 	mean error(validation) = 0.962193839924633
900 7.299815430585925 	mean error(validation) 

### 4.6. Perform an online solve

In [ ]:
online_mu = (10.0, 0.01)
reduced_problem.set_mu(online_mu)
reduced_solution = reduced_problem.solve()
print(reduced_solution)
plot(reduced_solution, reduced_problem=reduced_problem)

In [ ]:
net.eval()

with torch.no_grad():
    online_mu_nn = torch.tensor(online_mu)
    # NOTE: Change output_normalization accordingly for the different losses
    reduced_solution_nn = net.solve(online_mu_nn, input_normalization, output_normalization=None)
    print(reduced_solution_nn)
    plot(reduced_solution_nn, reduced_problem=reduced_problem)

net.train()

In [ ]:
errors, error_plot = NN.plot_error(net, test_mu, input_normalization)
print(np.mean(errors))
error_plot

### 4.7. Perform an error analysis

In [ ]:
reduction_method.initialize_testing_set(100, SCM=100)
reduction_method.error_analysis(filename="error_analysis")

### 4.8. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis(filename="speedup_analysis")

## 5. Assignments
1. Consider the following domain instead

<img src="data/graetz_2.png" width="70%"/>
Edit the domain defined in the generate_mesh notebook in the _data_ folder accordingly. Then, consider two additional parameters $\mu_2 \in [0.5, 1.5]$ and $\mu_3 \in [0.5, 1.5]$ and change the Graetz class to account for the following boundary conditions:
    * boundaries $\Gamma_{o, 1} \cup \Gamma_{o, 5} \cup \Gamma_{o, 6}$ are kept at zero temperature,
    * boundaries $\Gamma_{o, 2}(\mu_0) \cup \Gamma_{o, 4}(\mu_0)$ are kept at temperature $\mu_2$,
    * boundaries $\Gamma_{o, 7}(\mu_0) \cup \Gamma_{o, 8}(\mu_0)$ are kept at temperature $\mu_3$.
    
Discuss the performance of the reduction in terms of errors and speedups. _Suggestion: make sure also to change the mesh, subdomains and boundaries filenames in the mesh generation notebook before saving it to file. Moreover, for every new notebook copy change the value returned by the name() method of the Graetz class to avoid conflicts between this notebook and your copy._
